In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 开始使用Vertex AI模型注册表
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/get_started_with_model_registry.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_registry%2Fget_started_with_model_registry.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab企业版中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/get_started_with_model_registry.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Vertex AI Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/get_started_with_model_registry.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Vertex AI SDK在Vertex AI模型注册表中创建和注册多个模型版本。

了解更多关于[Vertex AI模型注册表](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)。

### 目标

在本教程中，您将学习如何使用Vertex AI模型注册表来创建和登记模型的多个版本。

本教程使用以下Vertex AI服务和资源：

- Vertex AI模型注册表
- Vertex AI模型资源
- Vertex AI终端资源

执行的步骤包括：

- 创建并注册第一个版本的模型到Vertex AI模型注册表。
- 创建并注册第二个版本的模型到Vertex AI模型注册表。
- 更新默认的模型版本。
- 删除模型版本。
- 重新训练下一个模型版本。

### 模型

本教程使用了来自TensorFlow Hub的预训练图像分类模型，该模型是在ImageNet数据集上训练的。

了解更多关于[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

成本

本教程使用 Google Cloud 的收费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

开始吧

### GPU运行时

*如果有这个选项，请确保在GPU运行时下运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU。**

### 为 Python 安装 Vertex AI SDK 和其他所需的软件包

In [ ]:
# Install the packages
USER=''
! pip3 install {USER} --upgrade --quiet google-cloud-aiplatform \
                                        tensorflow==2.15.1 \
                                        tensorflow-hub

### 重新启动运行时（仅适用于Colab）

为了使用新安装的包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。请等待完成后再继续进行下一步。⚠️</b>
</div>

### 验证您的笔记本环境（仅适用于Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

设置Google Cloud项目信息并为Python初始化Vertex AI SDK

要开始使用Vertex AI，您必须拥有现有的Google Cloud项目，并启用Vertex AI API。了解更多关于设置项目和开发环境的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

LOCATION = "us-central1"  # @param {type: "string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的桶还不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $LOCATION $BUCKET_URI

### 导入库

In [ ]:
import os

import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像，并为虚拟机（VM）实例分配的GPU数量。例如，要使用一个具有4个Nvidia Tesla T4 GPU的GPU容器映像分配给每个VM，您可以指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4)

否则，请指定`(None, None)`以使用一个在CPU上运行的容器映像。

了解有关[您所在位置的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：在2.3版本之前的TF中，GPU支持在此教程中加载自定义模型时会失败。这是一个已知问题，并已在TF 2.3中修复。这是由生成在serving函数中的静态图操作引起的。如果在您自己的自定义模型中遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于预测的预构建Docker容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2.13".replace(".", "-")

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)


DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量 `DEPLOY_COMPUTE` 设置为配置用于预测的VM所使用的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB 内存。
     - `n1-highmem`: 每个vCPU 6.5GB 内存
     - `n1-highcpu`: 每个vCPU 0.9 GB 内存
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96 \]中的数量

*注：您还可以使用n2和e2的机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
DEPLOY_COMPUTE = "n1-standard-4"
print("Train machine type", DEPLOY_COMPUTE)

## 模型注册表简介

Vertex AI模型注册表服务为您提供了创建和组合多个模型资源版本的能力。通过版本控制，您可以：

- 跨版本跟踪模型的渊源。
- 设置默认版本，即被认可的版本。
- 其他服务操作，如部署，会自动使用默认版本，无需特别指定版本。
- 为各个版本分配别名，如开发、暂存和生产。

## 从 TensorFlow Hub 获取预训练模型

为了演示目的，本教程使用了从 TensorFlow Hub（TFHub）获取的预训练模型的两个版本，然后将它们上传到 Vertex AI 模型注册表中，作为相同模型资源的版本 1 和版本 2。一旦您拥有了 Vertex AI 模型资源，模型版本就可以部署到 Vertex AI 终端资源上。

### 下载预训练模型

首先，您可以从 TensorFlow Hub 下载这两个版本的预训练模型。这些模型会以 TF.Keras 层的形式下载。为了最终确定模型，在这个示例中，您要创建一个带有下载的 TFHub 模型作为层的 `Sequential()` 模型，并指定模型的输入形状。

In [ ]:
tfhub_model_v1 = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model_v1.build([None, 224, 224, 3])

tfhub_model_v1.summary()

In [ ]:
tfhub_model_v2 = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model_v2.build([None, 128, 128, 3])

tfhub_model_v2.summary()

### 保存模型文件

在这一点上，模型的两个版本都在内存中。接下来，将模型的文件保存到云存储位置。

In [ ]:
MODEL_V1_DIR = BUCKET_URI + "/model/v1"
tfhub_model_v1.save(MODEL_V1_DIR)

MODEL_V2_DIR = BUCKET_URI + "/model/v2"
tfhub_model_v2.save(MODEL_V2_DIR)

### 将TensorFlow Hub模型的版本1上传到Vertex AI模型资源

接下来，将模型的第一个版本（`MODEL_DIR_V1`）作为模型资源上传到Vertex AI模型注册表中，并附加以下参数：

- `is_default_version`：此是否为模型资源的默认版本。
- `version_ailiases`：用户定义的模型版本的备用别名列表，例如`production`。
- `version_description`：模型版本的用户描述。

当在Vertex AI模型注册表中创建第一个模型版本时，属性`version_id`会自动设置为1。

In [ ]:
model_v1 = aip.Model.upload(
    display_name="example_",
    artifact_uri=MODEL_V1_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    is_default_version=True,
    version_aliases=["v1"],
    version_description="This is the first version of the model",
)

print(model_v1)

将TensorFlow Hub模型的第2版上传到Vertex AI模型资源

接下来，将第二个版本的模型（`MODEL_DIR_V1`）作为模型资源上传到Vertex AI模型注册表中，并添加以下附加参数：

- `parent_model`：要将此模型添加为下一个模型版本的现有模型资源。
- `is_default_version`：这是否是模型资源的默认版本。在这个例子中，您从该模型的第一个版本默认更改为第二个版本。
- `version_ailiases`：用户定义的用于模型版本的备用别名列表，例如`production`。
- `version_description`：模型版本的用户描述。

在Vertex AI模型注册表中创建后续模型版本时，属性`version_id`会自动递增。在这个例子中，它设置为2（第二版）。

In [ ]:
model_v2 = aip.Model.upload(
    display_name="example_",
    artifact_uri=MODEL_V2_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    parent_model=model_v1.resource_name,
    is_default_version=True,
    version_aliases=["v2"],
    version_description="This is the second version of the model",
)

print(model_v2)

获取所有父模型的版本

接下来，使用`version_registry.list_versions()`列出父模型的所有版本。

In [ ]:
versions = model_v1.versioning_registry.list_versions()
for version in versions:
    print(version)

获取一个非父模型的所有版本

您重复相同的操作，但这次使用非父模型。如您所见，无论您指定的模型是父模型还是非父模型，您都可以获取到所有版本。

In [ ]:
versions = model_v2.versioning_registry.list_versions()
for version in versions:
    print(version)

### 列出模型资源

与模型资源相关的其余 Vertex AI SDK 方法会自动使用模型资源的默认版本。

接下来，使用`list()`方法并陦用过滤器来获取您之前创建的模型资源。在这个例子中，有两个版本。第二个版本被设置为默认版本，所以`list()`方法只会返回版本2（默认版本）的信息。

In [ ]:
models = aip.Model.list(filter="display_name=example_")
print("Number of models:", len(models))
print("Version ID:", models[0].version_id)

model = models[0]

### 更改默认模型版本

接下来，使用`versioning_registry.add_version_aliases()`方法来更改模型资源的默认模型版本，使用以下参数：

- `version`：指定此操作适用于哪个版本，由version_id指定。
- `new_aliases`：要分配给指定版本的附加别名。

在这个例子中，您可以引用特殊别名`default`来将此模型（版本1）更改为默认模型。

接下来，使用`list()`方法，该方法现在返回模型的版本1，而不是版本2。

In [ ]:
model_v2.versioning_registry.add_version_aliases(new_aliases=["default"], version="1")

models = aip.Model.list(filter="display_name=example_")
print("Number of models:", len(models))
print("Version ID:", models[0].version_id)

model = models[0]

## 创建端点资源

您可以使用`Endpoint.create()`方法创建端点资源。最低限度，您需要为端点指定显示名称。可选地，您可以指定项目和位置（区域）；否则，设置将继承您使用`init()`方法初始化Vertex AI SDK时设置的值。

在此示例中，指定了以下参数：

- `display_name`：端点资源的人类可读名称。
- `project`：您的项目ID。
- `location`：您的区域。
- `labels`：（可选）以键/值对形式指定的端点的用户定义元数据。

此方法返回一个端点对象。

了解更多关于[Vertex AI端点](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aip.Endpoint.create(
    display_name="example_",
    project=PROJECT_ID,
    location=LOCATION,
    labels={"your_key": "your_value"},
)

print(endpoint)

将模型资源部署到端点资源。

您可以将一个或多个Vertex AI模型资源实例部署到同一个端点。每个部署的Vertex AI模型资源都有自己的用于服务二进制文件的部署容器。

*注意:* 对于此示例，您在上传模型构件到Vertex AI模型资源的上一步中指定了TFHub模型的部署容器。

### 部署单个端点资源

在下一个示例中，您将将Vertex AI模型资源的默认版本部署到一个Vertex AI端点资源。Vertex AI模型资源已经为其定义了部署容器镜像。要部署，您需要指定以下额外的配置设置：

- 机器类型。
- （如果有的话）GPU的类型和数量。
- 静态、手动或自动缩放的VM实例。

在此示例中，您将使用最少指定参数部署模型，如下：

- `model`: 模型资源。
- `deployed_model_displayed_name`: 部署的模型实例的易读名称。
- `machine_type`: 每个VM实例的机器类型。

由于资源的规定要求，这可能需要几分钟时间来提供。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_",
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

获取部署模型的信息

您可以从端点资源配置数据 `gca_resource.deployed_models` 中获取部署模型的部署设置。在本示例中，只部署了一个模型，因此引用下标 `[0]`。

In [ ]:
endpoint.gca_resource.deployed_models[0]

### 从端点资源中撤销模型资源

当模型资源部署到端点资源时，部署的模型资源实例会被分配一个ID，通常被称为部署的模型ID。

您可以使用`undeploy()`方法撤销特定的模型资源实例，参数如下:

- `deployed_model_id`: 分配给部署模型的ID。

In [ ]:
deployed_model_id = endpoint.gca_resource.deployed_models[0].id
print(deployed_model_id)

endpoint.undeploy(deployed_model_id)

### 删除模型版本

要从 Vertex AI 模型注册表中删除一个模型的版本，您可以使用 `versioning_registry.delete_version()` 方法，并使用以下参数：

- `version`: 可以是 `version_id` 或 `version_alias` 中的别名。

*注意：* 您不能删除默认模型版本。

In [ ]:
# Deletes a specific model version, as long as it isn't the default version.
model.versioning_registry.delete_version("2")

versions = model.versioning_registry.list_versions()
for version in versions:
    print(version)

删除模型资源

当您删除一个模型资源时，所有相关版本都会被删除。

In [ ]:
model.delete()

### 为Vertex AI Training指定模型版本

在使用Vertex AI Training服务训练模型时，您可以指定经过训练的模型为Vertex AI模型注册表中的父模型，或者父模型的一个版本，通过向`run()`方法添加以下附加参数：

- `parent_model`：要将此模型添加为下一个模型版本的现有模型资源。
- `is_default_version`：是否为模型资源的默认版本。
- `model_version_ailiases`：用户定义的模型版本的替代别名列表，例如`production`。
- `model_version_description`：模型版本的用户描述。

以下是一个示例用法：

```
# 该作业上传my-training-job模型的新版本，非默认版本
job = aiplatform.CustomTrainingJob(
   display_name="my-training-job",
   script_path="training_script.py",
   container_uri="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-2:latest",
   requirements=["gcsfs==0.7.1"],
   model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-2:latest",
)

model = job.run(
   my_dataset,
   replica_count=1,
   machine_type="n1-standard-4",
   accelerator_type='NVIDIA_TESLA_T4',
   accelerator_count=1,
   model_id='my-model'
   model_version_aliases=["myalias"],
   model_version_description="My description",
   is_default_version=False,
)

```

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}

# Undeploy the models and delete the endpoint resources
try:
    endpoint.undeploy_all()
    endpoint.delete()
except Exception as e:
    print(e)